# Creating Features Quiz
Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) run the cells below to read dataset and build body length feature
# 3) write code to answer the quiz questions 
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.sql.functions import col, lit, concat, udf
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/13 19:29:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/13 19:29:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/01/13 19:29:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/01/13 19:29:14 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Body Length Feature

In [5]:
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [6]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [7]:
df.head()

Traceback (most recent call last):                                  (0 + 1) / 1]
  File "/usr/local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/local/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

# Question 1
Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [8]:
# TODO: write your code to answer question 1
df.select("BodyLength").where(df.Id == 1112).show()

+----------+
|BodyLength|
+----------+
|        63|
+----------+



# Question 2
Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [9]:
# TODO: write your code to answer question 2
# Make a new col named "Desc" that concated by col "Title" and "Body"
# df.select(concat("Title", lit(' '), "Body")).show()
df = df.withColumn("Desc", concat("Title", lit(' '), "Body"))

In [10]:
# Tokenize words2
regexTokenizer = RegexTokenizer(inputCol="Desc", outputCol="words2", pattern="\\W")
df = regexTokenizer.transform(df)
# Count words2
df = df.withColumn("DescLength", body_length(df.words2))

In [11]:
df.select("DescLength").where(df.Id == 5123).show()

+----------+
|DescLength|
+----------+
|       135|
+----------+



# Create a Vector
Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [12]:
# TODO: write your code to create this vector
assembler = VectorAssembler(inputCols=["DescLength"], outputCol="DescVec")
df = assembler.transform(df)

# Question 3
Using the Normalizer method what's the normalized value for question Id = 512?

In [13]:
# TODO: write your code to answer question 3
normalizer= Normalizer(inputCol="DescVec", outputCol="DescVecNormalizer")
df = normalizer.transform(df)

In [14]:
df.select("DescVecNormalizer").where(df.Id == 512).show()

+-----------------+
|DescVecNormalizer|
+-----------------+
|            [1.0]|
+-----------------+



# Question 4
Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [15]:
# TODO: write your code to answer question 4
standardScaler = StandardScaler(inputCol="DescVec", outputCol="DescStandardScalar", withMean=True, withStd=True)
standardScalerModel = standardScaler.fit(df)
df = standardScalerModel.transform(df)

In [16]:
df.select("DescStandardScalar").where(df.Id == 512).show()

+--------------------+
|  DescStandardScalar|
+--------------------+
|[-0.6417314460998...|
+--------------------+



# Question 5
Using the MinMAxScaler method what's the normalized value for question Id = 512?

In [17]:
# TODO: write your code to answer question 5
minMaxScaler = MinMaxScaler(inputCol="DescVec", outputCol="DescMinMaxScalar")
minMaxScalerModel = minMaxScaler.fit(df)
df = minMaxScalerModel.transform(df)

In [18]:
df.select("DescMinMaxScalar").where(df.Id == 512).show()

+--------------------+
|    DescMinMaxScalar|
+--------------------+
|[0.00624833820792...|
+--------------------+

